In [1]:
import numpy as np
import pandas as pd
import sys, os, time
main_folder = './../'
if main_folder not in sys.path:
    sys.path.append(main_folder)
from evolution import DifferentialEvolution
from glob       import glob
from tqdm       import tqdm

import dirs
from utils      import get_solution, ProgressBar, write_log
import matplotlib.pyplot as plt 

progbar = ProgressBar(elapsed_time=True)

### Parameters

In [2]:
# Competition Parameters
funcId      = 6
dim         = 10
numRuns     = 51
targetError = 1e-8

# Algorithm Parameters
pop_size = 30
percMaxEvals = 1
maxEvals = 'auto'
maxGenerations = None
fitness_clusters = None
crossover = 'binonial'
mutation = 'best'
n_diff = 1
lambda_mutation = .5
opposition = False
input_data_filepath = './../input_data'
active = False

In [3]:
maxEvals = maxEvals if percMaxEvals is None else percMaxEvals*10000*dim
run_tag = '[' + '_'.join([str(funcId), 
                          str(dim), 
                          str(numRuns),
                          str(pop_size), 
                          str(percMaxEvals),                                                     
                          '{}'.format(lambda_mutation), 
                          str(opposition),
                          str(fitness_clusters)
                         ])  + ']'
run_tag

'[6_10_51_30_1_0.5_False_None]'

In [4]:
alg = DifferentialEvolution(dim=dim, func_id=funcId, pop_size=pop_size, crossover=crossover, 
        opposition=opposition, mutation=mutation, lambda_mutation=.5, fitness_clusters=fitness_clusters)

output_filepath = os.path.join(dirs.results, str(alg))
if not os.path.exists(output_filepath):
    os.makedirs(output_filepath)
output_filepath = output_filepath + '/' + run_tag

print ('Writing log at ', output_filepath + 'log.tsv')
write_log(filepath=output_filepath + 'log.tsv', 
          mode="w+", 
          text='[{}]\trun\telapsedTime\tsuccessRate\n'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())))  

Writing log at  ./results/DE/best/1/bin/[6_10_51_30_1_0.5_False_None]log.tsv


In [5]:
hist = pd.DataFrame()
arr_runs = range(numRuns)
solution = get_solution(funcId, dim, input_data_filepath=input_data_filepath)
successRate = 0
for idx_run in arr_runs:
    progbar.update_progress(idx_run/float(len(arr_runs)))
    
    # Differential Evolution
    alg = DifferentialEvolution(dim=dim, func_id=funcId, pop_size=pop_size, crossover=crossover, 
        opposition=opposition, mutation=mutation, lambda_mutation=.5, fitness_clusters=fitness_clusters)
    
    errorHist, fitnessHist = alg.optimize(target=solution, max_f_evals=maxEvals, target_error=targetError, verbose=True)        
    bestError = errorHist.iloc[-1,:].min()
    errorHist["run"] = np.ones(errorHist.shape[0], dtype=int)*idx_run
    hist = pd.concat([hist, errorHist], ignore_index=False)    
    if bestError <= targetError:
        print ("Target error achieved with error {}".format(bestError))
        successRate += 1
    
    write_log(filepath=output_filepath + 'log.tsv', 
              text='[{}]\t{}/{}\t{}\t{}\n'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()), 
                                              idx_run+1,
                                              len(arr_runs),
                                              progbar.get_elapsed_time(),
                                              successRate)
             )
    
hist.index.name = 'generation'
hist.to_csv(output_filepath + 'hist.tsv', sep='\t')
progbar.update_progress(1)

[0:00:00][----------] 0.00% Optimization ended due to max fitness evals (max = 100000, curr = 96780)

#Generations:	3224
#FitnessEvals:	96780
Mean Fitness:	601.0689
Best Fitness:	601.0689
Solution:	600.0000
Diff:		1.0689
[0:02:38][----------] 1.96% Optimization ended due to max fitness evals (max = 100000, curr = 96780)

#Generations:	3224
#FitnessEvals:	96780
Mean Fitness:	600.1646
Best Fitness:	600.1646
Solution:	600.0000
Diff:		0.1646
[0:05:18][----------] 3.92% Optimization ended due to max fitness evals (max = 100000, curr = 96780)

#Generations:	3224
#FitnessEvals:	96780
Mean Fitness:	600.2820
Best Fitness:	600.2820
Solution:	600.0000
Diff:		0.2820
[0:07:59][#---------] 5.88% Optimization ended due to max fitness evals (max = 100000, curr = 96780)

#Generations:	3224
#FitnessEvals:	96780
Mean Fitness:	602.3946
Best Fitness:	602.3946
Solution:	600.0000
Diff:		2.3946
[0:10:39][#---------] 7.84% Optimization ended due to max fitness evals (max = 100000, curr = 96780)

#Generations:	

[1:34:28][#######---] 72.55% 
#Generations:	531
#FitnessEvals:	15990
Mean Fitness:	600.0000
Best Fitness:	600.0000
Solution:	600.0000
Diff:		0.0000
Target error achieved with error 4.909907147521153e-09
[1:34:54][#######---] 74.51% Optimization ended due to max fitness evals (max = 100000, curr = 96780)

#Generations:	3224
#FitnessEvals:	96780
Mean Fitness:	600.4643
Best Fitness:	600.4643
Solution:	600.0000
Diff:		0.4643
[1:37:34][########--] 76.47% Optimization ended due to max fitness evals (max = 100000, curr = 96780)

#Generations:	3224
#FitnessEvals:	96780
Mean Fitness:	600.1316
Best Fitness:	600.1316
Solution:	600.0000
Diff:		0.1316
[1:40:15][########--] 78.43% Optimization ended due to max fitness evals (max = 100000, curr = 96780)

#Generations:	3224
#FitnessEvals:	96780
Mean Fitness:	600.0000
Best Fitness:	600.0000
Solution:	600.0000
Diff:		0.0000
[1:42:55][########--] 80.39% Optimization ended due to max fitness evals (max = 100000, curr = 96780)

#Generations:	3224
#FitnessE

In [6]:
print ("Elapsed Time: ", progbar.get_elapsed_time())

Elapsed Time:  2:06:22


In [7]:
errorHist.tail()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,run
generation,,,,,,,,,,,,,,,,,,,,,
3219,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,...,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,50
3220,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,...,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,50
3221,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,...,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,50
3222,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,...,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,50
3223,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,...,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,50


In [137]:
arr_error = list()
arr_best_error = list()
for run in hist['run'].unique():
    df_run = hist[hist['run'] == run].drop(['run'], axis=1) 
    bestError = df_run.iloc[-1,:].min()    
    arr_best_error.append(bestError)    
    print ('Best error for run {}: {}'.format(run, bestError))
    arr_error.append(np.array(df_run.min(axis=1)))        
arr_error = np.array(arr_error)



Best error for run 0: 1.068851230092264
Best error for run 1: 0.16455414225242748
Best error for run 2: 0.28199484563629085
Best error for run 3: 2.3945754284394525
Best error for run 4: 1.0742234568089088
Best error for run 5: 0.019722970769862513
Best error for run 6: 0.8966402521689361
Best error for run 7: 0.5189611499401963
Best error for run 8: 0.9912501037156289
Best error for run 9: 0.17952962569825104
Best error for run 10: 1.595460876990387
Best error for run 11: 2.824208002039086e-05
Best error for run 12: 0.2820513299199092
Best error for run 13: 5.233914635027759e-09
Best error for run 14: 0.11872748247367326
Best error for run 15: 0.06763015217802604
Best error for run 16: 0.11570166654109926
Best error for run 17: 0.13159490245050165
Best error for run 18: 0.09873810806709571
Best error for run 19: 3.15165389355775
Best error for run 20: 0.0020572891648953373
Best error for run 21: 1.8386794664181707
Best error for run 22: 0.4593048508351103
Best error for run 23: 1.5819

In [138]:
df_sumup = pd.DataFrame(columns=['algorithm', '#Fc', 'D', 'percFES'] + ['run_' + str(i) for i in np.arange(1,numRuns+1,1)] + ['mean', 'elapsed_time', 'evals'])
df_sumup.loc[df_sumup.shape[0]] = [str(alg), funcId, dim, percMaxEvals] + arr_best_error + [np.mean(arr_best_error), progbar.get_elapsed_time(), alg.fitnessEvals]
print ('Saving df_sumup to ', output_filepath + 'df_sumup.tsv')
df_sumup.to_csv(output_filepath + 'df_sumup.tsv', sep='\t')
df_sumup

Saving df_sumup to  ./results/DE/best/1/bin/[6_10_51_30_1_0.5_False_None]df_sumup.tsv


,algorithm,#Fc,D,percFES,run_1,run_2,run_3,run_4,run_5,run_6,...,run_45,run_46,run_47,run_48,run_49,run_50,run_51,mean,elapsed_time,evals
0,DE/best/1/bin,6,10,1,1.068851,0.164554,0.281995,2.394575,1.074223,0.019723,...,1.5,0.518965,0.966364,9.089263e-09,0.008872,8.368715e-09,0.000032,0.781152,2:06:22,96780


In [ ]:
# fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,10))
# label = 'Success Rate: {}%\nMin Error Fitness: {}'.format(100*(successRate/float(numRuns)), arr_error.min())
# # label = ''
# ax.errorbar(np.arange(alg.generations), arr_error.mean(axis=0), yerr=arr_error.std(axis=0), label=label)
# ax2 = ax.twinx()
# ax2.plot(np.arange(alg.generations), 20*np.log(arr_error.mean(axis=0)), label='Mean Min Error (dB)', color='red')
# ax2.set_ylabel('Mean Minimum Error (dB)')

# ax.grid(True)
# ax.set_ylabel('Mean Minimum Error')
# ax.legend()
# ax.set_xlabel('Generation')
# ax.set_title('Error Fitness Function {} (Solution = {})'.format(funcId, solution))
# plt.savefig(output_filepath + 'mean_error_evolution.jpeg', bbox_inches = 'tight')